In [ ]:
import os
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

from dotenv import load_dotenv

load_dotenv()

llm_name = "gpt-4-1106-preview"
os.environ["OPENAI_API_KEY"]  # you should see the api key if the .env file is loaded correctly

In [44]:
# Replace with persist_directory location from 01_load_hummingbot_docs.ipnynb
persist_directory = "/Users/feng/Code/ai-assistant/vector_stores"

embedding = OpenAIEmbeddings()

vectordb = Chroma(
    collection_name="hummingbot_documentation",
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [45]:
from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# Build prompt
template = """Use the following pieces of context that are part of the Hummingbot Documentation to answer the user question. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

prompt = ChatPromptTemplate.from_template(template=template)
model = ChatOpenAI(model_name=llm_name, temperature=0)
output_parser = StrOutputParser()

chain = prompt | model

In [46]:
query = """Can you explain me how to use the pure market making strategy?
"""

In [47]:
from langchain.schema.runnable import RunnableMap

retriever = vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10)

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [48]:
chain.invoke(
    input={
        "question": query,
    }
)

'The Pure Market Making strategy in Hummingbot involves placing both buy and sell limit orders on a single exchange (the maker side). If one side of the order is filled, you would typically not hedge this on another exchange, which is a key difference from the cross-exchange market making strategy. You can configure the strategy by setting parameters such as the order amount, the spread, and how often to refresh orders, among others, using the `pure_market_making_config_map.py` file. Thanks for asking!'

In [49]:
class Strategy(BaseModel):
    """Strategy created by the LLM for the user."""
    code: str = Field(description="Provide the python code for the strategy.")
    summary: str = Field(description="Provide a concise summary of what the strategy is doing.")

In [50]:

strategy_extraction_function = [
    convert_pydantic_to_openai_function(Strategy)
]
tagging_model = model.bind(
    functions=strategy_extraction_function,
    function_call={"name":"Strategy"}
)

In [51]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

tagging_chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | StrOutputParser() | tagging_model | JsonOutputFunctionsParser()

In [52]:
result = tagging_chain.invoke(
    input={
        "question": query,
    }
)

In [53]:
print(result["summary"])

This Python code snippet demonstrates how to configure and initialize a Pure Market Making strategy using Hummingbot. It sets up the strategy parameters such as order amount, bid and ask spreads, minimum spread, and order refresh time. It then initializes the market trading pair tuple with the exchange and trading pair information. Finally, it creates an instance of the PureMarketMakingStrategy with the configured parameters and starts the strategy. The strategy is then run in a loop, simulating the continuous operation of a trading bot.


In [54]:
print(result["code"])

import pandas as pd
import numpy as np
import time
from hummingbot.strategy.market_trading_pair_tuple import MarketTradingPairTuple
from hummingbot.strategy.pure_market_making import PureMarketMakingStrategy
from hummingbot.strategy.pure_market_making.pure_market_making_config_map import pure_market_making_config_map

# Example configuration
pure_market_making_config_map['order_amount'].value = 1.0
pure_market_making_config_map['bid_spread'].value = 0.01
pure_market_making_config_map['ask_spread'].value = 0.01
pure_market_making_config_map['minimum_spread'].value = 0.005
pure_market_making_config_map['order_refresh_time'].value = 30.0

# Initialize the market trading pair tuple
market_info = MarketTradingPairTuple(exchange='binance',
                                     base_asset='BTC',
                                     quote_asset='USDT',
                                     trading_pair='BTC-USDT')

# Initialize the strategy
strategy = PureMarketMakingStrategy(market_info=market_